## Loading Data

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-12-27 16:13:07--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.207, 173.194.216.207, 173.194.217.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   207MB/s    in 0.3s    

2024-12-27 16:13:07 (207 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Extract Data


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalize the pixel values to the range [0, 1]
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Flow data from the training directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Flow data from the validation directory
validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Load ResNet50

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    weights='imagenet'
)


pretrained_model.trainable = False


model = models.Sequential([
    pretrained_model,
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## Compile

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

## Train

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 278s 4s/step - accuracy: 0.5293 - loss: 0.7358 - val_accuracy: 0.5790 - val_loss: 0.6774
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 272s 4s/step - accuracy: 0.5505 - loss: 0.6903 - val_accuracy: 0.5470 - val_loss: 0.6790
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 321s 4s/step - accuracy: 0.5848 - loss: 0.6721 - val_accuracy: 0.5170 - val_loss: 0.6827
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 322s 4s/step - accuracy: 0.5628 - loss: 0.6769 - val_accuracy: 0.5770 - val_loss: 0.6727
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 272s 4s/step - accuracy: 0.5574 - loss: 0.6774 - val_accuracy: 0.6060 - val_loss: 0.6654
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 320s 4s/step - accuracy: 0.5913 - loss: 0.6637 - val_accuracy: 0.5970 - val_loss: 0.6632
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 321s 4s/step - accuracy: 0.5814 - loss: 0.6724 - val_accuracy: 0.6050 - val_loss: 0.6639
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 270s 4s/step - accuracy: 0.6023 - loss: 0.6627 - val_accuracy: 0.6050 - val_loss: 0.

## Fine Tuning


In [37]:
pretrained_model.trainable = True


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


history_fine = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 913s 14s/step - accuracy: 0.7303 - loss: 2.2959 - val_accuracy: 0.5000 - val_loss: 1.5815
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 851s 14s/step - accuracy: 0.9802 - loss: 0.0772 - val_accuracy: 0.5000 - val_loss: 11.9863
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 846s 13s/step - accuracy: 0.9972 - loss: 0.0165 - val_accuracy: 0.5000 - val_loss: 13.0061
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 919s 14s/step - accuracy: 0.9976 - loss: 0.0101 - val_accuracy: 0.5000 - val_loss: 13.6974
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 922s 14s/step - accuracy: 0.9957 - loss: 0.0123 - val_accuracy: 0.5000 - val_loss: 17.6056


## Evaluate Model

In [38]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy}")

#Save trained model
model.save('cats_dogs_classifier.h5')

32/32 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.4963 - loss: 17.6366


Validation Accuracy: 0.5
